# Analyse de Data Drift avec Evidently

In [ ]:
from copy import deepcopy
from os import environ
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer
from requests import get, Response
from hashlib import sha256
from tqdm.notebook import tqdm
from zipfile import ZipFile
from IPython.display import display, Markdown
import pandas as pd
import missingno as msno
import seaborn as sns
import numpy as np
from IPython.display import clear_output
from lightgbm import LGBMClassifier
from lightgbm import LGBMClassifier

import re
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
# Added imports for the new transformer
from scipy import stats
from random import sample

from lightgbm import LGBMClassifier
from utils.image_inverter import save
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate, GridSearchCV, \
    ParameterGrid
from mlflow import set_tracking_uri, log_metrics, log_figure
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

environ['CUDA_VISIBLE_DEVICES'] = '-1'
_cache_folder = Path('~/.cache/gn_p7').expanduser()
_cache_folder.mkdir(parents=True, exist_ok=True)

_ds_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip'

graph_folder: Path = Path("./graphs")
random_state: int = 42


def save_figure(figure: plt.Figure, folder: str, figure_name: str) -> None:
    folder = graph_folder / folder
    folder.mkdir(parents=True, exist_ok=True)
    save(figure, folder / f'{figure_name}.png', close=True)


def download(url: str) -> Path:
    url_id: str = sha256(url.encode('utf-8')).hexdigest()
    local_path: Path = _cache_folder / url_id
    local_path.parent.mkdir(parents=True, exist_ok=True)
    if not local_path.exists():
        tmp_path: Path = _cache_folder / (url_id + '.tmp')
        res: Response = get(url, stream=True)
        with tmp_path.open('wb') as f, tqdm(
                total=int(res.headers.get('content-length')),
                desc=f'Downloading {url}',
                unit_scale=True) as q:
            for chunk in res.iter_content(chunk_size=8192):
                q.update(len(chunk))
                f.write(chunk)
        tmp_path.replace(local_path)
    return local_path


def download_zip_archive(url: str) -> Path:
    """Download a zip archive, extract it then return the folder containing its content"""
    archive_path: Path = download(url)
    archive_folder: Path = Path(archive_path.as_posix() + '.dir')

    if not archive_folder.exists():
        print(f'Extracting archive {url}...', flush=True)
        archive_temp: Path = Path(archive_path.as_posix() + '.tmp')
        archive_temp.mkdir(parents=True, exist_ok=True)
        archive: ZipFile = ZipFile(archive_path)
        archive.extractall(path=archive_temp)
        archive_temp.replace(archive_folder)
        print(f'Extracting archive {url}...done', flush=True)

    return archive_folder


datasets: dict[str, pd.DataFrame] = {}


def get_dataset(name: str) -> pd.DataFrame:
    folder = download_zip_archive(_ds_url)
    if not name.endswith('.csv'):
        name = f'{name}.csv'
    try:
        return datasets[name]
    except KeyError:
        try:
            _df = pd.read_csv(folder / name)
        except FileNotFoundError:
            display(Markdown(f'# ERROR: Dataset {name!r} not found, available datasets are:\n' + '\n'.join(
                f'- {p.name}' for p in sorted(folder.iterdir(), key=(lambda x: x.name.lower())))))
            raise KeyError(name) from None
        else:
            datasets[name] = _df
            return _df.copy()


In [ ]:
import pandas as pd
from evidently import Report

from zipfile import ZipFile
from pathlib import Path

from evidently.presets import DataDriftPreset

# Chemin vers le zip téléchargé (adapte si nécessaire)
archive_folder = Path('..')

# Chargement des datasets
app_train_df = get_dataset( 'application_train.csv')
app_test_df = get_dataset('application_test.csv')

# Hypothèse : app_train est la référence (données d'entraînement)
# et app_test simule les nouvelles données en production
reference_data = app_train_df.sample(n=20000, random_state=42)
current_data = app_test_df.sample(n=20000, random_state=42)

print("Données de référence chargées:", reference_data.shape)
print("Données courantes chargées:", current_data.shape)

(run := Report(metrics=[DataDriftPreset(), ]).run(reference_data=reference_data, current_data=current_data)).save_html(
    'GHOUL_Nadir_4_Tableau_HTML_data_drift_evidently.html')

run